In [1]:
%run LanguageModel.py

In [2]:
%run DataLoader.py

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_dl = DataLoader('train', ('de', 'en'), max_length = 50, device = device)

In [5]:
val_dl = DataLoader('dev', ('de', 'en'), languageModels = train_dl.languageModels, max_length = 50, device = device)

In [6]:
%run seq2seq.py

In [7]:
hidden_size = 256
lm1 = train_dl.languageModels[train_dl.languages[0]]
lm2 = train_dl.languageModels[train_dl.languages[1]]
s2s = seq2seq(lm1.n_tokens, hidden_size, lm2.n_tokens, device, rnn_type='gru', bidirectional=False, attention = 'global', beam_width = 3, max_length=52)

In [8]:
def loss_metric(input, output, ground_truth, nll):
    return nll / ground_truth[0].size(0)

def perplexity(input, output, ground_truth, nll):
    nll /= ground_truth[0].size(0)
    return math.exp(nll)

In [9]:
%run ScorePrinter.py

In [10]:
def train_epochs(epochs, print_every=1000):
    n_iters = len(train_dl)
    score_printer = ScorePrinter("Training", [('NLL', loss_metric),('Perplexity', perplexity)])
    
    for epoch in range(1, epochs+1):
        score_printer.startEpoch(epoch)
        idx_permutation = np.random.permutation(len(train_dl))[:5]
        for i, index in enumerate(idx_permutation):
            input_tensor, target_tensor = train_dl.tensorsFromPos(index)

            loss, output_sentence = s2s.train(input_tensor, target_tensor)
            score_printer.update(input_tensor, target_tensor, output_sentence, loss)
            
            if (i + 1) % print_every == 0:
                score_printer.printAvg(print_every)
                validate(print_every)
        
        score_printer.endEpoch(epoch)
        score_printer.printAvg(len(train_dl))
        validate()

In [11]:
import numpy as np
import math

def validate(n = None):
    score_printer = ScorePrinter("Validation", [('NLL', loss_metric),('Perplexity', perplexity)])
    n = n or len(val_dl)
    idx_permutation_val = np.random.permutation(len(val_dl))[:n]
    for j, val_index in enumerate(idx_permutation_val):
        input_tensor_val, target_tensor_val = val_dl.tensorsFromPos(val_index)
        loss, output_sentence = s2s.evaluate(input_tensor_val, target_tensor_val)
        score_printer.update(input_tensor_val, target_tensor_val, output_sentence, loss)
    score_printer.printAvg(showCount = False)

In [12]:
train_epochs(50, print_every=2)



Epoch 1 started
 
[Training] 2 examples /  NLL: 10.750194506211713 Perplexity: 46639.49919487725  
[Validation]   NLL: 4.451227152122642 Perplexity: 281.4541318035665  
[Training] 4 examples /  NLL: 10.75334873756805 Perplexity: 47334.77648904289  
[Validation]   NLL: 7.074014663696289 Perplexity: 1201.4698463932891 

Epoch 1 ended
 
[Training] 5 examples /  NLL: 10.649097942996335 Perplexity: 43423.94980523963  
[Validation]   NLL: 7.118533897794688 Perplexity: 1870.6216521397696 

Epoch 2 started
 
[Training] 7 examples /  NLL: 10.62491100545921 Perplexity: 43510.92769911227  
[Validation]   NLL: 7.362708409627279 Perplexity: 1598.8186288648476  
[Training] 9 examples /  NLL: 10.781485039452035 Perplexity: 53662.37057706742  
[Validation]   NLL: 6.243568817774454 Perplexity: 751.5104746875284 

Epoch 2 ended
 
[Training] 10 examples /  NLL: 10.388134585433589 Perplexity: 48390.33118555551  
[Validation]   NLL: 6.537252128267712 Perplexity: 1173.1405481827132 

Epoch 3 started
 
[Tr

KeyboardInterrupt: 